In [11]:
!pip install google-cloud-secret-manager==2.0.0

     |████████████████████████████████| 88 kB 4.0 MB/s eta 0:00:011


In [17]:
def access_secret_version(name):
    """
    Access the payload for the given secret version if one exists. The version
    can be a version number as a string (e.g. "5") or an alias (e.g. "latest").
    """

    # Import the Secret Manager client library.
    from google.cloud import secretmanager

    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()
    
    # Access the secret version.
    response = client.access_secret_version(request={"name": name})

    # Print the secret payload.
    #
    # WARNING: Do not print the secret in a production environment - this
    # snippet is showing how to access the secret material.
    return response.payload.data.decode("UTF-8")

In [19]:
api_key=access_secret_version(name='projects/468034158681/secrets/talentcards_api/versions/1')

In [1]:
import json

import requests
from requests.auth import HTTPBasicAuth

try:
    from urllib import quote_plus
except ImportError:
    from urllib.parse import quote_plus

In [4]:
class TalentCardsApi(object):
    def __init__(self, api_url='https://www.talentcards.io/api/v1/', api_key):
        self.api_url = api_url
        self.auth = HTTPBasicAuth(api_key, '')
        
        def get(self, params={}):
            params_list = []

            for param, val in params.items():
                params_list.append(quote_plus(str(param)) + ':' + quote_plus(str(val), safe='@'))

            params_list.sort()
            get_params = ','.join(params_list)

            resp = requests.get('{}/{}/{}'.format(self.api_url, get_params),
                            auth=self.auth)
            result = json.loads(resp.text)

            return result
        
        def users(self, search_term=None):
            if search_term is None:
                return self.get('users')
            elif type(search_term) == int or search_term.isdigit(): 
                return self.get('users', {'id': search_term})
            elif '@' in search_term:
                return self.get('users', {'email': search_term})
            else:
                return self.get('users', {'username': search_term})


In [ ]:

talentcards=TalentCardsApi()